In [ ]:
%%capture
%pip install jsonschema networkx numpy scipy pythreejs
%pip install compas>=2.8 --no-deps
%pip install compas_notebook>=0.8.2 --no-deps
%pip install compas_model --no-deps
%pip install compas_dem --no-deps

# DEM of a Cross Vault

## Block Data

In [1]:
import pathlib

from compas.files import OBJ

FILE = pathlib.Path.cwd() / "data" / "crossvault.obj"

obj = OBJ(FILE)
obj.read()

## Block Meshes

In [2]:
from compas.datastructures import Mesh

meshes = []
for name in obj.objects:
    vertices, faces = obj.objects[name]
    mesh: Mesh = Mesh.from_vertices_and_faces(vertices, faces)
    mesh.scale(0.025, 0.025, 0.025)
    mesh.name = name
    meshes.append(mesh)

## Block Model

In [3]:
from compas_dem.models import BlockModel

model = BlockModel.from_boxes(meshes)

## Contacts

In [4]:
# model.compute_contacts(tolerance=1e-3, minimum_area=1e-2)

## Supports

In [5]:
from compas_dem.elements import BlockElement

element: BlockElement

for element in model.elements():
    if model.graph.degree(element.graphnode) == 1:
        element.is_support = True

## Visualisation

In [7]:
from compas_notebook import Viewer
from compas_notebook.config import Config

config = Config()
config.view.camera.position = [0, -13, 3]
config.view.camera.target = [0, 0, 3]

viewer = Viewer(config=config)
viewer.scene.add(model)
viewer.show()